<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EA%B3%BC%EC%A0%95/(%EB%85%B8%EC%9D%B4)C%EC%A4%91%EB%B6%84%EB%A5%98/(%EB%85%B8%EC%9D%B4)_%EC%9B%90%EB%B3%B8C%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A4%91%EB%B6%84%EB%A5%98_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 60.0 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 9.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595735 sha256=9910b60faa935fab6d3379e8192df6c9a6cd8b78dc054599c5cbe5fa9a16c721
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 8.3 MB/s 
     |████████████████████████████████| 769 kB 6.8 MB/s 
     |████████████████████████████████| 3.0 MB 41.0 MB/s 
     |████████████████████████████████| 895 kB 35.2 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-27y1n93_
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/p

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본   광주재똥   산업분류자동화  'Colab Notebooks'  'My Drive'


In [25]:
# spell check한 data
import pandas as pd
df=pd.read_csv('/mydrive/산업분류자동화/dataset/spell_check/C_spell_check.csv', sep = ",", encoding = "UTF-8")
ts=pd.read_csv('/mydrive/산업분류자동화/답안 작성용 파일.csv', encoding = "euc-kr")

# 증분한 data
#import pandas as pd
#df=pd.read_csv('/mydrive/산업분류자동화/dataset/inc/df_R_inc.csv', sep = ",", encoding = "UTF-8")
#ts=pd.read_csv('/mydrive/산업분류자동화/답안 작성용 파일.csv', encoding = "euc-kr")

In [26]:
df=df.reset_index(drop=True)

## 데이터 전처리

In [27]:
df.head(10)

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000006,C,29,291,철 절삭. 용접 카프라 배관자재
1,id_0000008,C,10,107,쌀을 가지고 가공하여 떡 제조
2,id_0000011,C,10,102,"멸치 입고, 가공"
3,id_0000021,C,10,107,쌀을 가지고 가루로 분쇄하여 떡을 쪄서 백설기 꿀떡 절편 판매
4,id_0000027,C,22,221,"고무 절단가공 개스킷, 다이아 프레임"
5,id_0000030,C,29,292,"Resim 재료 구입, 세미앗세이"
6,id_0000044,C,25,259,금속재료 주문에 의한 절삭가공 공작기계
7,id_0000053,C,22,222,공장에서 다른 사업체의 요청에 의해 부품 제조
8,id_0000056,C,32,320,소파 제조사업장에서 주문에 의해 소파 제조
9,id_0000081,C,14,141,원단을 가지고 재단 봉제하여 아동복


In [28]:
# 결측값 확인
df.isnull().sum()

AI_id      0
digit_1    0
digit_2    0
digit_3    0
text       0
dtype: int64

In [29]:
df.shape

(105192, 5)

In [33]:
df['digit_2'].value_counts()

25    17065
10    14576
29    12087
22     5738
28     5524
14     5417
13     5287
33     4584
18     4177
34     3380
26     3306
32     3275
27     3207
20     3029
23     2873
30     2772
24     2215
17     1754
16     1724
15     1379
31     1032
11      387
21      313
19       88
12        3
Name: digit_2, dtype: int64

In [37]:
a = df[['digit_2','digit_3']].value_counts().sort_index()

In [38]:
a.to_csv('C_jung_so.csv')

In [99]:
import pandas as pd
df=pd.read_csv('/mydrive/산업분류자동화/dataset/spell_check/C_spell_check.csv', sep = ",", encoding = "UTF-8")

In [100]:
df1 = df.loc[df['digit_2'] < 14]
df2 = df.loc[(df['digit_2']>13)&(df['digit_2']<20)]
df3 = df.loc[(df['digit_2']>19)&(df['digit_2']<25)]
df4 = df.loc[(df['digit_2']>24)&(df['digit_2']<29)]
df5 = df.loc[df['digit_2']>28]

In [101]:
df = df1

In [102]:
df['digit_3'].value_counts().sort_index()

101     729
102     985
103    1177
104     769
105      64
106    1353
107    9214
108     285
111     250
112     137
120       3
131     474
132    2819
133     217
134     965
139     812
Name: digit_3, dtype: int64



#### **추후 과제: 분류가 현재 매우 불균형함 / 정규화를 이용해서 균형을 맞추면 정확도가 올라가는지 확인 필요**


In [103]:
df.loc[(df['digit_3'] == 101), 'digit_3'] = 0  
df.loc[(df['digit_3'] == 102), 'digit_3'] = 1  
df.loc[(df['digit_3'] == 103), 'digit_3'] = 2  
df.loc[(df['digit_3'] == 104), 'digit_3'] = 3  
df.loc[(df['digit_3'] == 105), 'digit_3'] = 4
df.loc[(df['digit_3'] == 106), 'digit_3'] = 5
df.loc[(df['digit_3'] == 107), 'digit_3'] = 6
df.loc[(df['digit_3'] == 108), 'digit_3'] = 7
df.loc[(df['digit_3'] == 111), 'digit_3'] = 8
df.loc[(df['digit_3'] == 112), 'digit_3'] = 9
df.loc[(df['digit_3'] == 120), 'digit_3'] = 10
df.loc[(df['digit_3'] == 131), 'digit_3'] = 11
df.loc[(df['digit_3'] == 132), 'digit_3'] = 12
df.loc[(df['digit_3'] == 133), 'digit_3'] = 13
df.loc[(df['digit_3'] == 134), 'digit_3'] = 14
df.loc[(df['digit_3'] == 139), 'digit_3'] = 15
#df.loc[(df['digit_3'] == 529), 'digit_3'] = 16
#df.loc[(df['digit_3'] == 529), 'digit_3'] = 17
#df.loc[(df['digit_3'] == 529), 'digit_3'] = 18

In [104]:
data_list = []
for q, label in zip(df['text'], df['digit_3'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['쌀을 가지고 가공하여 떡 제조', '6'], ['멸치 입고, 가공 ', '1'], ['쌀을 가지고 가루로 분쇄하여 떡을 쪄서 백설기 꿀떡 절편 판매', '6'], ['구입한 소금과 조기로 염장, 세척, 엮걸이 굴비 제조', '1'], ['벼를 도정 쌀', '5']]


## Train data & test data

In [105]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

# KoBERT 입력 데이터로 만들기

In [106]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [107]:
# Setting parameters
max_len = 64
batch_size = 64 #32,16,8로 감소시키기
warmup_ratio = 0.1
num_epochs = 10 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [108]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [109]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [110]:
data_train[0]

(array([   2, 1258, 6117, 6305, 6903, 3807, 7119, 7096,  763, 6971, 1342,
         768, 4427, 5561, 6117, 1943, 1788, 6136,    3,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(19, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 3)

In [111]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습 모델 만들기

In [112]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=16,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [113]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [114]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/285 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.7858846187591553 train acc 0.078125
epoch 1 batch id 201 loss 0.2951544225215912 train acc 0.7199160447761194
epoch 1 train acc 0.7856209150326797


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 9 test acc 0.9120861235119048


  0%|          | 0/285 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.15290099382400513 train acc 0.96875
epoch 10 batch id 201 loss 0.009528584778308868 train acc 0.9916044776119403
epoch 10 train acc 0.9917214912280702


  0%|          | 0/32 [00:00<?, ?it/s]

epoch 10 test acc 0.9125744047619048


In [115]:
#학습 모델 저장
torch.save(model, 'C_ori_jung_model_1_epoch10.pt')